## Notes 
- For the moment the model is only tokenized with the `text_body` and not with the `text_title`
- The evalutation training metric is the `accuracy`. Is it the best metric for an ordered variable ? (classify 4 instead of 5 is less important than classify 1 instead of 5)
- for the moment we do not monitor the evaluation metric during fine-tuning. What does it mean ? Is it useful ? 


In [45]:
model_fr_name = 'camembert-base'
model_en_name = 'roberta-base'
model_traduction = 'Helsinki-NLP/opus-mt-fr-en'
dataset_name = 'amazon_reviews_multi'

In [1]:
from datasets import load_dataset

dataset = load_dataset(dataset_name)

No config specified, defaulting to: amazon_reviews_multi/all_languages


Extracting data files:   0%|          | 0/6 [00:00<?, ?it/s]

Extracting data files:   0%|          | 0/6 [00:00<?, ?it/s]

Extracting data files:   0%|          | 0/6 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/1200000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/30000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/30000 [00:00<?, ? examples/s]

Dataset amazon_reviews_multi downloaded and prepared to C:/Users/mathi/.cache/huggingface/datasets/amazon_reviews_multi/all_languages/1.0.0/724e94f4b0c6c405ce7e476a6c5ef4f87db30799ad49f765094cf9770e0f7609. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [31]:
dataset_fr_train = dataset['train'].filter(lambda exemple : exemple['language'] == 'fr')
dataset_fr_test = dataset['test'].filter(lambda exemple : exemple['language'] == 'fr')
dataset_en_train = dataset['train'].filter(lambda exemple : exemple['language'] == 'en')
dataset_en_test = dataset['test'].filter(lambda exemple : exemple['language'] == 'en')

dataset_fr_train

Loading cached processed dataset at C:\Users\mathi\.cache\huggingface\datasets\amazon_reviews_multi\all_languages\1.0.0\724e94f4b0c6c405ce7e476a6c5ef4f87db30799ad49f765094cf9770e0f7609\cache-2489db10e829ee0e.arrow
Loading cached processed dataset at C:\Users\mathi\.cache\huggingface\datasets\amazon_reviews_multi\all_languages\1.0.0\724e94f4b0c6c405ce7e476a6c5ef4f87db30799ad49f765094cf9770e0f7609\cache-0b1824e786950535.arrow
Loading cached processed dataset at C:\Users\mathi\.cache\huggingface\datasets\amazon_reviews_multi\all_languages\1.0.0\724e94f4b0c6c405ce7e476a6c5ef4f87db30799ad49f765094cf9770e0f7609\cache-9f25e2bdda2b899b.arrow


Filter:   0%|          | 0/30000 [00:00<?, ? examples/s]

Dataset({
    features: ['review_id', 'product_id', 'reviewer_id', 'stars', 'review_body', 'review_title', 'language', 'product_category'],
    num_rows: 200000
})

In [75]:
def prepare_dataset_for_classification(dataset, language):
    dataset = dataset.filter(lambda example : example['language'] == language)
    dataset = dataset.remove_columns(["review_id","product_id", "reviewer_id",
    "review_title","language","product_category"])
    dataset = dataset.rename_column("review_body", "text")
    dataset = dataset.rename_column("stars", "label")
    return dataset

dataset_fr_train = prepare_dataset_for_classification(dataset['train'], 'fr')
dataset_fr_validation = prepare_dataset_for_classification(dataset['validation'], 'fr')

Loading cached processed dataset at C:\Users\mathi\.cache\huggingface\datasets\amazon_reviews_multi\all_languages\1.0.0\724e94f4b0c6c405ce7e476a6c5ef4f87db30799ad49f765094cf9770e0f7609\cache-ae218842b6d78ba4.arrow


Filter:   0%|          | 0/30000 [00:00<?, ? examples/s]

In [76]:
from transformers import AutoTokenizer

tokenizer_fr = AutoTokenizer.from_pretrained(model_fr_name)
# tokenizer_en = AutoTokenizer.from_pretrained(model_en_name)

def tokenize_function(examples):
    return tokenizer(examples["text"], 
                     padding="max_length", truncation=True)


tokenized_fr_train = dataset_fr_train.map(tokenize_function, batched=True)
tokenized_fr_validation = dataset_fr_validation.map(tokenize_function, batched=True)
# tokenized_en_train = dataset_en_train.map(tokenize_function, batched=True)
# tokenized_en_test = dataset_en_test.map(tokenize_function, batched=True)

Could not locate the tokenizer configuration file, will try to use the model config instead.
loading configuration file config.json from cache at C:\Users\mathi/.cache\huggingface\hub\models--camembert-base\snapshots\3f452b6e5a89b0e6c828c9bba2642bc577086eae\config.json
Model config CamembertConfig {
  "_name_or_path": "camembert-base",
  "architectures": [
    "CamembertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 5,
  "classifier_dropout": null,
  "eos_token_id": 6,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "camembert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_past": true,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.25.1",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 32005
}

loading file sentencepiece.

Map:   0%|          | 0/200000 [00:00<?, ? examples/s]

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

In [83]:
small_train_fr_dataset = tokenized_fr_train.shuffle(seed=42).select(range(200))
small_validation_fr_dataset = tokenized_fr_validation.shuffle(seed=42).select(range(200))

Loading cached shuffled indices for dataset at C:\Users\mathi\.cache\huggingface\datasets\amazon_reviews_multi\all_languages\1.0.0\724e94f4b0c6c405ce7e476a6c5ef4f87db30799ad49f765094cf9770e0f7609\cache-c9837626a7207fba.arrow
Loading cached shuffled indices for dataset at C:\Users\mathi\.cache\huggingface\datasets\amazon_reviews_multi\all_languages\1.0.0\724e94f4b0c6c405ce7e476a6c5ef4f87db30799ad49f765094cf9770e0f7609\cache-ba8a65b343f25686.arrow


In [78]:
from transformers import AutoModelForSequenceClassification

model_fr = AutoModelForSequenceClassification.from_pretrained(model_fr_name, num_labels=5)

loading configuration file config.json from cache at C:\Users\mathi/.cache\huggingface\hub\models--camembert-base\snapshots\3f452b6e5a89b0e6c828c9bba2642bc577086eae\config.json
Model config CamembertConfig {
  "_name_or_path": "camembert-base",
  "architectures": [
    "CamembertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 5,
  "classifier_dropout": null,
  "eos_token_id": 6,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4
  },
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "camembert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_past": true,
  "pad_token_id": 1,
  "position_embedding_type": "absolut

In [79]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(output_dir="test_trainer_fr")
# If we want to monitor the evaluation metric during fine-tuning
# training_args = TrainingArguments(output_dir="test_trainer_fr", evaluation_strategy="epoch")

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [80]:
import numpy as np
import evaluate

metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [84]:
trainer = Trainer(
    model=model_fr,
    args=training_args,
    train_dataset=small_train_fr_dataset,
    eval_dataset=small_validation_fr_dataset,
    compute_metrics=compute_metrics,
)
trainer.train()

The following columns in the training set don't have a corresponding argument in `CamembertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `CamembertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 200
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 75
  Number of trainable parameters = 110625797


RuntimeError: [enforce fail at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\c10\core\impl\alloc_cpu.cpp:72] data. DefaultCPUAllocator: not enough memory: you tried to allocate 100663296 bytes.

## Traduction

SOhelp : https://stackoverflow.com/questions/70043467/how-to-run-huggingface-helsinki-nlp-models

In [55]:
# Try 1 of traduction --> Does not work 
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer_trad = AutoTokenizer.from_pretrained(model_traduction)
model_trad = AutoModelForSeq2SeqLM.from_pretrained(model_traduction)

inp = "Je m'appelle Mathieu et je vis à Lausanne"
input_ids = tokenizer_trad(inp, return_tensors="pt").input_ids
outputs = model_trad.generate(input_ids=input_ids, num_return_sequences=3)
print("Generated:", tokenizer.batch_decode(outputs, skip_special_tokens=True))

loading configuration file config.json from cache at C:\Users\mathi/.cache\huggingface\hub\models--Helsinki-NLP--opus-mt-fr-en\snapshots\49463f1706007cb314a942296b77a6483e6f6953\config.json
Model config MarianConfig {
  "_name_or_path": "Helsinki-NLP/opus-mt-fr-en",
  "_num_labels": 3,
  "activation_dropout": 0.0,
  "activation_function": "swish",
  "add_bias_logits": false,
  "add_final_layer_norm": false,
  "architectures": [
    "MarianMTModel"
  ],
  "attention_dropout": 0.0,
  "bad_words_ids": [
    [
      59513
    ]
  ],
  "bos_token_id": 0,
  "classif_dropout": 0.0,
  "classifier_dropout": 0.0,
  "d_model": 512,
  "decoder_attention_heads": 8,
  "decoder_ffn_dim": 2048,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 6,
  "decoder_start_token_id": 59513,
  "decoder_vocab_size": 59514,
  "dropout": 0.1,
  "encoder_attention_heads": 8,
  "encoder_ffn_dim": 2048,
  "encoder_layerdrop": 0.0,
  "encoder_layers": 6,
  "eos_token_id": 0,
  "forced_eos_token_id": 0,
  "gradient_checkp

Generated: ['politiquesux suréclosion', 'politiquesux suréclosion', 'politiquesux.éclosion']


In [57]:
# Try 2 that works
from transformers import pipeline

translator = pipeline("translation", model=model_traduction)
translator("Ce cours est produit par Hugging Face.")


loading configuration file config.json from cache at C:\Users\mathi/.cache\huggingface\hub\models--Helsinki-NLP--opus-mt-fr-en\snapshots\49463f1706007cb314a942296b77a6483e6f6953\config.json
Model config MarianConfig {
  "_name_or_path": "Helsinki-NLP/opus-mt-fr-en",
  "_num_labels": 3,
  "activation_dropout": 0.0,
  "activation_function": "swish",
  "add_bias_logits": false,
  "add_final_layer_norm": false,
  "architectures": [
    "MarianMTModel"
  ],
  "attention_dropout": 0.0,
  "bad_words_ids": [
    [
      59513
    ]
  ],
  "bos_token_id": 0,
  "classif_dropout": 0.0,
  "classifier_dropout": 0.0,
  "d_model": 512,
  "decoder_attention_heads": 8,
  "decoder_ffn_dim": 2048,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 6,
  "decoder_start_token_id": 59513,
  "decoder_vocab_size": 59514,
  "dropout": 0.1,
  "encoder_attention_heads": 8,
  "encoder_ffn_dim": 2048,
  "encoder_layerdrop": 0.0,
  "encoder_layers": 6,
  "eos_token_id": 0,
  "forced_eos_token_id": 0,
  "gradient_checkp

[{'translation_text': 'This course is produced by Hugging Face.'}]

In [59]:
def translate(example):
    example["traduction"] = translator(example['review_body'])
    return example

translated_dataset = small_train_fr_dataset.map(translate)

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

KeyboardInterrupt: 